In [1]:
# default_exp packaging

# Simplified conda & pip packaging & deployment


# Determine minimal dependencies

* pipreqs: `pipreqs scidev --use-local`
* [pigar](https://github.com/damnever/pigar): `pigar scidev`
* TODO: add utility for conda

Additions to `Makefile`
```
local_release: local_pip_release local_conda_release
	nbdev_bump_version
    
local_pip_release: clean
    python setup.py bdist_wheel upload -r local
    
local_conda_release:
    curl -4 -XPUT -u"${ARTIFACTORY_USER}":"${ARTIFACTORY_PASSWORD}" "${bamboo_artifactory_url}/${CHANNEL}/linux-64/" -T "${OUTPUT_PATH}"
```

# Artifactory Environment Variables for conda

* export ARTIFACTORY_USER=..
* export ARTIFACTORY_PASSWORD=..
* export ARTIFACTORY_URL=ndartifactory.jfrog.io
* export ARTIFACTORY_CONDA_CHANNEL=conda-local

In [45]:
# export

import os
import subprocess
import sys
from pathlib import Path
from typing import List

from nbdev.export import Config

In [ ]:
%load_ext autoreload
%autoreload 2

In [4]:
# Need to generate meta.yaml from settings.ini

In [24]:
# export


def determine_dependencies(generated_pip_file_name: str = "requirements-generated.txt"):
    try:
        pass
    except:
        print("Pigar dependency is not installed - not able to determine dependencies")
        return
    lib_path = Config().path("lib_path")
    project_root = lib_path.resolve().parent
    command = "pigar"
    args = ["-p", generated_pip_file_name, "-P", lib_path]

    output = subprocess.run(
        [sys.executable, "-m", command, *(str(i).strip() for i in args)],
        stderr=subprocess.PIPE,
        stdout=subprocess.PIPE,
        cwd=project_root,
        env=None,
        universal_newlines=True,
    )

    output_code = output.returncode

    output.stdout
    err = output.stderr

    if output_code != 0:
        raise (EnvironmentError(err))

In [27]:
generated_reqs_path = os.path.join(
    Path(".").resolve().parent, "requirements-generated.txt"
)
if os.path.exists(generated_reqs_path):
    os.remove(generated_reqs_path)
assert not os.path.exists(generated_reqs_path)
determine_dependencies()
assert os.path.exists(generated_reqs_path)
os.remove(generated_reqs_path)

# Requirement.txt Manipulation

> Read pip requirements file and convert to a structure that can be used to transform that output to a different format.

For more information see here:

https://www.python.org/dev/peps/pep-0440/#version-specifiers

In [66]:
test_lines = (
    "fastcore == 1.3.19",
    "\n",
    "#",
    "nbformat >= 5.0.8",
    "# scidev/nb_lint.py: 10,11,12",
    "nbqa ~= 0.5.6",
    "nbqa <=0.5.6",
)

In [76]:
def reqs_lines_to_sep_str(req_lines: List[str], sep: str = " "):
    return " ".join(
        [
            l.replace(" ", "").strip()
            for l in req_lines
            if not l.startswith("#") and len(l.strip()) > 0
        ]
    )

In [77]:
assert (
    "fastcore==1.3.19 nbformat>=5.0.8 nbqa~=0.5.6 "
    "nbqa<=0.5.6" == reqs_lines_to_sep_str(test_lines)
)

In [78]:
def reqs_file_to_sep_str(pip_reqs_path: Path) -> str:
    with open(pip_reqs_path, "r") as pip_reqs_file:
        lines = pip_reqs_file.readlines()
    return reqs_lines_to_sep_str(lines)

In [79]:
determine_dependencies()
reqs_str = reqs_file_to_sep_str(generated_reqs_path)
os.remove(generated_reqs_path)

In [80]:
reqs_str

'fastcore==1.3.19 nbdev==1.1.12 nbformat==5.0.8 nbqa==0.5.6'

In [82]:
def update_requirements(output_filename: str = "settings-generated.ini"):
    config = ConfigParser(delimiters=["="])
    settings_path = os.path.join(Config().path("lib_path"), "settings.ini")
    config.read(settings_path)

    generated_reqs_path = os.path.join(
        Path(".").resolve().parent, "requirements-generated.txt"
    )
    determine_dependencies()
    reqs_str = reqs_file_to_sep_str(generated_reqs_path)
    os.remove(generated_reqs_path)

    os.path.join(Path(".").resolve().parent, output_filename)
    config.set("DEFAULT", "requirements", reqs_str)

    with open(new_settings_path, "w") as configfile:
        config.write(configfile)

In [24]:
# Writing our configuration file to 'example.ini'

In [ ]:
# Populate requirements in settings.ini
# requirements =

In [ ]:
# curl -U nexus_eut_svc:AKCp8hzCxcyh1Uh9s4LMNZKmEHY1V4jwUhfqRsEUvdvMNZKhyncwt8MTtzwaq2rZkhx6dxW3e -T ~/envs/discovery/conda-bld/linux-64/scidev-0.0.1-0.tar.bz2 "https://ndartifactory.jfrog.io/artifactory/ra_conda_dev/linux-64"

# curl -4 -XPUT
# curl -4 -XPUT "https://nexus_eut_svc:AKCp8hzCxcyh1Uh9s4LMNZKmEHY1V4jwUhfqRsEUvdvMNZKhyncwt8MTtzwaq2rZkhx6dxW3e@ndartifactory.jfrog.io/artifactory/ra_conda_dev/linux-64/" -T /home/jovyan/envs/discovery/conda-bld/linux-64/scidev-0.0.1-0.tar.bz2

1. Add minimal requires to settings.ini
2. Test local_release_pip
3. Create meta.yaml from settings.ini
4. Test local_conda_release
5. Test release